## Estimate the camera gain and offset from experimental data

This notebook presents one approach to estimating the average gain and offset of a camera directly from experimental data. This can be useful if you just want a quick and approximately accurate estimate, or you are tasked with analyzing data that someone else gave you and they don't provide this information. In order to work it relies on the presence of a constant (on the 200 frame time scale) non-uniform background in the experimental data. 

Reference:
* [Kothe et al, Histochemistry and Cell Biology, 2014](https://doi.org/10.1007/s00418-014-1211-4)


### Configuring the directory
Create an empty directory somewhere on your computer and tell Python to go to that directory.

In [ ]:
import matplotlib.pyplot as pyplot
import numpy
import os

import storm_analysis.sa_library.datareader as dataReader

os.chdir("/home/hbabcock/Data/storm_analysis/jy_testing/")
print(os.getcwd())

numpy.random.seed(1)

### Create simulated data

In [ ]:
import storm_analysis.jupyter_examples.est_cam_params as est_cam_params

# Create localizations for simulation.
est_cam_params.createLocalizations()

# Create simulated movie (200 frames)
est_cam_params.createMovie(gain = 2.0, offset = 100.0)

Display an image

In [ ]:
import storm_analysis.sa_library.datareader as datareader

frame = datareader.inferReader("test.tif").loadAFrame(5).astype(numpy.float64)

pyplot.figure(figsize = (6, 6))
pyplot.imshow(frame, interpolation = 'nearest', cmap = "gray")
pyplot.show()

### Load movie and calculate mean and variance for each pixel

In [ ]:
def calcMeanVar(movie_name):
    with dataReader.inferReader(movie_name) as dr:
        [w,h,l] = dr.filmSize()
        if(l>200):
            l = 200
        n = numpy.zeros((h,w), dtype = numpy.int64)
        nn = numpy.zeros((h,w), dtype = numpy.int64)
        for i in range(l):
            im = dr.loadAFrame(i)
            im = im.astype(numpy.int64)
            n = n + im
            nn = nn + im*im
        
    mean = n/float(l)
    var = nn/float(l) - mean*mean
    
    return [mean, var]


In [ ]:
[mean, var] = calcMeanVar("test.tif")

### Plot data with estimated fit

In this case we know what the actual camera values are so we can just use them. In general the easiest approach is to do the fit by eye, adjusting the gain and offset values to give you a line that goes through the cluster of points at the bottom of the mean vs variance graph. In Kothe et al. the authors estimated the fit using the [RANSAC](https://en.wikipedia.org/wiki/Random_sample_consensus) algorithm.

In [ ]:
# You may have to adjust these depending on your data.
x_max = 150
x_min = 100

x = numpy.array([x_min, x_max])

# In this function 100.0 is the offset and 2.0 is the camera gain.
y = (x - 100.0)*2.0
    
pyplot.scatter(mean,var,s=1)
pyplot.plot(x,y,color = "black")
pyplot.xlim(x_min,x_max)

# You may have to adjust the y range depending on your data.
pyplot.ylim(0,200)
pyplot.show()
    